In [1]:
import concurrent.futures
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pprint import pprint


domain = "https://www.dangquangwatch.vn"


def get_soup(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
    }
    with requests.Session() as session:
        response = session.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "lxml")
    return soup


def get_product_info(url):
    soup = get_soup(url)
    title_name = soup.select_one(".product-subname").text.split(" - ")[0].split(" ")

    name = " ".join(title_name[:-1])

    price = soup.select_one(".price").text.replace("\xa0₫", "")
    product_code = title_name[-1]

    details = {}
    table = soup.select_one(".detail_product_right .depro_right_content").find_all(
        class_="item"
    )
    for row in table:
        list_p = row.find_all(name="p")
        first_name = list_p[0].get_text().strip()
        second_name = list_p[1].get_text().strip()
        details[first_name] = second_name
    return {
        "Mã sản phẩm": product_code,
        "Giá tiền": price,
        **details,
    }


def crawl_data_from_web(all_links):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(get_product_info, all_links)
    data_frame_watchs = pd.DataFrame(results)
    return data_frame_watchs


all_page = [
    f"https://www.dangquangwatch.vn/sp/dong-ho-chinh-hang.html?&page={idx_page}"
    for idx_page in range(1, 125)
]
all_links = [
    (domain + a["href"]) for url in all_page for a in get_soup(url).select("a.p_info")
]

data_frame_watchs = crawl_data_from_web(all_links)

data_frame_watchs


KeyboardInterrupt: 